In [1]:
import os, sys
import numpy as np
from matplotlib import pyplot as plt

import pandas as pd

from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import OneHotEncoder

from keras.models import Sequential, Model
# from keras.layers import Input, Embedding, Reshape, merge, LSTM, Bidirectional
# from keras.layers import TimeDistributed, Activation, SimpleRNN, GRU
from keras.layers.core import Flatten, Dense, Dropout, Lambda
#from keras.regularizers import l2, activity_l2, l1, activity_l1
from keras.layers.normalization import BatchNormalization
from keras.optimizers import SGD, RMSprop, Adam

from sklearn.model_selection import train_test_split

from pre_process import *

%matplotlib inline

Using TensorFlow backend.


In [2]:
# Init some useful dirs
current_dir = os.getcwd()
DATA_HOME_DIR = current_dir+'/../data/'

## Data

In [3]:
pd.options.display.max_columns = None

In [4]:
cols = ['SK_ID_CURR',
        # Some columns that sound useful!
        'NAME_CONTRACT_TYPE',
        'CODE_GENDER',
        'FLAG_OWN_CAR',
        'FLAG_OWN_REALTY',
        'NAME_TYPE_SUITE',
        'NAME_INCOME_TYPE',
        'NAME_EDUCATION_TYPE',
        'NAME_FAMILY_STATUS',
        'NAME_HOUSING_TYPE',
        'DAYS_REGISTRATION',
        'OWN_CAR_AGE',
        # Positively correlated to the target ( top 10 )
        'DAYS_BIRTH',
        'REGION_RATING_CLIENT_W_CITY',
        'REGION_RATING_CLIENT',
        'DAYS_LAST_PHONE_CHANGE',
        'DAYS_ID_PUBLISH',
        'REG_CITY_NOT_WORK_CITY',
        'FLAG_EMP_PHONE',
        'REG_CITY_NOT_LIVE_CITY',
        'FLAG_DOCUMENT_3',
        # Negative correlated to the target (top 10)
        'ELEVATORS_AVG',
        'REGION_POPULATION_RELATIVE',
        'AMT_GOODS_PRICE',
        'FLOORSMAX_MODE',
        'FLOORSMAX_MEDI',
        'FLOORSMAX_AVG',
        'DAYS_EMPLOYED',
        'EXT_SOURCE_1',
        'EXT_SOURCE_3',
        'EXT_SOURCE_2'
        ]

In [5]:
df_train, df_test, y = load_train_test_data(DATA_HOME_DIR,in_cols=cols) 

In [6]:
df_train.shape

(307511, 31)

### Additional features

In [7]:
df_train, df_test = append_bureau_data(in_dir=DATA_HOME_DIR, df_train=df_train, df_test=df_test)

In [8]:
df_train, df_test = append_previous_applications(in_dir=DATA_HOME_DIR, df_train=df_train, df_test=df_test)

In [9]:
#df_train, df_test = append_pos_data(in_dir=DATA_HOME_DIR, df_train=df_train, df_test=df_test)

In [10]:
#df_train, df_test = append_credit_card_data(in_dir=DATA_HOME_DIR, df_train=df_train, df_test=df_test)

In [11]:
#df_train, df_test = append_installments_data(in_dir=DATA_HOME_DIR, df_train=df_train, df_test=df_test)

In [12]:
df_train.shape

(307511, 240)

### Numericals standardizing

In [13]:
numerical_feats = [
            f for f in df_train.columns if df_train[f].dtype == 'float64' or df_train[f].dtype == 'int64'
        ]


In [14]:
df_train_norm = normalize_numericals(df_train, numerical_feats)

In [15]:
df_test_norm = normalize_numericals(df_test, numerical_feats)

In [16]:
df_train_enc, df_test_enc = load_data_dummies(df_train_norm, df_test_norm)

In [17]:
df_train_enc.shape

(307511, 272)

In [18]:
df_test_enc.shape

(48744, 2750)

# Split data
TODO: cross folds

In [19]:
x_train, x_test, y_train, y_test = train_test_split( df_train_enc, y, test_size=0.2, random_state=42)

In [20]:
x_train.shape

(246008, 272)

In [21]:
y_train.shape

(246008,)

## Model Setup
Lets create a simple convolutional model with 2 layers and try to train that. This will be our baseline performance for any convolutional architecture we try 

In [22]:
in_layers = x_train.shape[1]
fd_layers = in_layers*2
learning_rate = 0.01

In [23]:
model = Sequential()
model.add(Dense(fd_layers, input_dim=in_layers, activation='relu'))
model.add(Dense(int((fd_layers/2)), activation='relu'))
model.add(BatchNormalization())
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer=Adam(lr=learning_rate), metrics=['accuracy'])

In [24]:
batch_size=64

In [25]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 544)               148512    
_________________________________________________________________
dense_2 (Dense)              (None, 272)               148240    
_________________________________________________________________
batch_normalization_1 (Batch (None, 272)               1088      
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 273       
Total params: 298,113
Trainable params: 297,569
Non-trainable params: 544
_________________________________________________________________


In [26]:
from roc_callback import *
roc_cb = roc_callback(training_data=(x_train, y_train),validation_data=(x_test, y_test))

In [27]:
model.fit(x_train, y_train, epochs=10, batch_size=batch_size, callbacks=[roc_cb])

Epoch 1/10
246008/246008 [==============================] - 24s 97us/step - loss: 0.2760 - acc: 0.9185
roc-auc: 0.6508 - roc-auc_val: 0.6534                                                                                                    
Epoch 2/10
246008/246008 [==============================] - 23s 94us/step - loss: 0.2710 - acc: 0.9192
roc-auc: 0.6587 - roc-auc_val: 0.6579                                                                                                    
Epoch 3/10
246008/246008 [==============================] - 23s 94us/step - loss: 0.2693 - acc: 0.9192
roc-auc: 0.6702 - roc-auc_val: 0.6697                                                                                                    
Epoch 4/10
246008/246008 [==============================] - 23s 95us/step - loss: 0.2681 - acc: 0.9191
roc-auc: 0.6719 - roc-auc_val: 0.6698                                                                                                    
Epoch 5/10
246008/246008 [==========

In [31]:
model.optimizer.lr = 0.01

In [32]:
model.fit(x_train, y_train, epochs=20, batch_size=batch_size, callbacks=[roc_cb])

Epoch 1/20
246008/246008 [==============================] - 17s 68us/step - loss: 0.2640 - acc: 0.9192
roc-auc: 0.6964 - roc-auc_val: 0.683                                                                                                    
Epoch 2/20
246008/246008 [==============================] - 17s 67us/step - loss: 0.2639 - acc: 0.9192
roc-auc: 0.6958 - roc-auc_val: 0.6825                                                                                                    
Epoch 3/20
246008/246008 [==============================] - 17s 67us/step - loss: 0.2637 - acc: 0.9192
roc-auc: 0.6992 - roc-auc_val: 0.6858                                                                                                    
Epoch 4/20
246008/246008 [==============================] - 17s 67us/step - loss: 0.2632 - acc: 0.9192
roc-auc: 0.7034 - roc-auc_val: 0.684                                                                                                    
Epoch 5/20
246008/246008 [============

In [33]:
model.optimizer.lr = 0.001
model.fit(x_train, y_train, epochs=20, batch_size=batch_size, callbacks=[roc_cb])

Epoch 1/20
246008/246008 [==============================] - 17s 67us/step - loss: 0.2591 - acc: 0.9193
roc-auc: 0.7182 - roc-auc_val: 0.6801                                                                                                    
Epoch 2/20
246008/246008 [==============================] - 17s 68us/step - loss: 0.2590 - acc: 0.9194
roc-auc: 0.7198 - roc-auc_val: 0.6814                                                                                                    
Epoch 3/20
246008/246008 [==============================] - 16s 67us/step - loss: 0.2588 - acc: 0.9195
roc-auc: 0.7222 - roc-auc_val: 0.6788                                                                                                    
Epoch 4/20
246008/246008 [==============================] - 17s 67us/step - loss: 0.2584 - acc: 0.9195
roc-auc: 0.7242 - roc-auc_val: 0.6776                                                                                                    
Epoch 5/20
246008/246008 [==========